# 2.5) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Import statements
Following packages are necessary:

In [1]:
import random
import utils
import ccall

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from unet import UNet
from unet import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from lastengen import LASTENSequence
from matplotlib import pyplot as plt

Segmentation Models: using `tf.keras` framework.


# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data
Import training images

In [3]:
#path_train = r'D:\LASTEN2\train'
#path_validation = r'D:\LASTEN2\validation'
#path_test = r'D:\LASTEN2\test'

path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/train'
path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/validation'
path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/test'

width = 384 #768
height = 384 #768

# Seeding
Seeds are set to ensure reproducible training.

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [5]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 384, 384, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 384, 384, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 384, 384, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [6]:
train_gen = LASTENSequence(path_train,
                           batch_size=1,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=1,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [7]:
save_path = "experiments/2_5_segmentation"

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)

callbacks = [logger, checker, checker_best, timelogger]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [8]:
model.fit(train_gen,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    #use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/100
160/160 [==============================] - 49s 309ms/step - loss: 0.7186 - accuracy: 0.9340 - iou_score: 0.1720 - val_loss: 0.7752 - val_accuracy: 0.9609 - val_iou_score: 0.1284
Epoch 2/100
160/160 [==============================] - 49s 305ms/step - loss: 0.3620 - accuracy: 0.9886 - iou_score: 0.4709 - val_loss: 0.3135 - val_accuracy: 0.9641 - val_iou_score: 0.5228
Epoch 3/100
160/160 [==============================] - 49s 306ms/step - loss: 0.3134 - accuracy: 0.9894 - iou_score: 0.5247 - val_loss: 0.2814 - val_accuracy: 0.9641 - val_iou_score: 0.5609
Epoch 4/100
160/160 [==============================] - 48s 303ms/step - loss: 0.3023 - accuracy: 0.9896 - iou_score: 0.5373 - val_loss: 0.3282 - val_accuracy: 0.9638 - val_iou_score: 0.5060
Epoch 5/100
160/160 [==============================] - 48s 303ms/step - loss: 0.3012 - accuracy: 0.9896 - iou_score: 0.5391 - val_loss: 0.2820 - val_accuracy: 0.9640 - val_iou_score: 0.5604
Epoch 6/100
160/160 [=============================

Epoch 44/100
160/160 [==============================] - 49s 305ms/step - loss: 0.2663 - accuracy: 0.9907 - iou_score: 0.5813 - val_loss: 0.2543 - val_accuracy: 0.9644 - val_iou_score: 0.5948
Epoch 45/100
160/160 [==============================] - 49s 306ms/step - loss: 0.2670 - accuracy: 0.9907 - iou_score: 0.5803 - val_loss: 0.2521 - val_accuracy: 0.9644 - val_iou_score: 0.5975
Epoch 46/100
160/160 [==============================] - 49s 306ms/step - loss: 0.2674 - accuracy: 0.9907 - iou_score: 0.5800 - val_loss: 0.2613 - val_accuracy: 0.9644 - val_iou_score: 0.5858
Epoch 47/100
160/160 [==============================] - 49s 305ms/step - loss: 0.2710 - accuracy: 0.9906 - iou_score: 0.5751 - val_loss: 0.2636 - val_accuracy: 0.9644 - val_iou_score: 0.5828
Epoch 48/100
160/160 [==============================] - 49s 306ms/step - loss: 0.2719 - accuracy: 0.9905 - iou_score: 0.5743 - val_loss: 0.2578 - val_accuracy: 0.9643 - val_iou_score: 0.5903
Epoch 49/100
160/160 [=======================

Epoch 87/100
160/160 [==============================] - 49s 305ms/step - loss: 0.2623 - accuracy: 0.9909 - iou_score: 0.5861 - val_loss: 0.2497 - val_accuracy: 0.9644 - val_iou_score: 0.6006
Epoch 88/100
160/160 [==============================] - 49s 305ms/step - loss: 0.2613 - accuracy: 0.9909 - iou_score: 0.5872 - val_loss: 0.2529 - val_accuracy: 0.9645 - val_iou_score: 0.5965
Epoch 89/100
160/160 [==============================] - 49s 304ms/step - loss: 0.2606 - accuracy: 0.9910 - iou_score: 0.5881 - val_loss: 0.2527 - val_accuracy: 0.9645 - val_iou_score: 0.5967
Epoch 90/100
160/160 [==============================] - 49s 306ms/step - loss: 0.2593 - accuracy: 0.9910 - iou_score: 0.5899 - val_loss: 0.2491 - val_accuracy: 0.9644 - val_iou_score: 0.6013
Epoch 91/100
160/160 [==============================] - 49s 304ms/step - loss: 0.2650 - accuracy: 0.9908 - iou_score: 0.5825 - val_loss: 0.2505 - val_accuracy: 0.9645 - val_iou_score: 0.5994
Epoch 92/100
160/160 [=======================

# Evaluation

In [9]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

In [10]:
test_gen = LASTENSequence(path_train,
                          batch_size=1,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)

In [11]:
X, y = test_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y_test.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred_test.png",y_pred[0,:,:,0], cmap="gray")

In [13]:
model.evaluate(test_gen)

160/160 [==============================] - 12s 78ms/step - loss: 0.2569 - accuracy: 0.9644 - iou_score: 0.5916


[0.25689977407455444, 0.9643614292144775, 0.5915509462356567]